# Spaceship Titanic

### 🚣‍♂️목적 
우주선 타이타닉 어떤 승객이 다른 차원으로 이동할지 예측

</br>

### 🧩특징 
우주 미스터리를 풀기 위해 여러분의 데이터 과학 기술이 필요한 2912년에 오신 것을 환영합니다.  
4광년 떨어진 곳에서 전송이 왔는데 상황이 좋지 않아요

우주선 타이타닉은 한 달 전에 발사된 성간 여객선입니다.  
거의 13,000명의 승객을 태운 이 배는 태양계에서 온 이민자들을  
근처의 별 주위를 도는 세 개의 새로이 거주할 수 있는 외계 행성으로 실어 나르는 처녀 항해를 시작했습니다.

첫 번째 목적지인 55호 암크리 E로 알파 센타우리를 도는 동안,  
조심성이 없는 우주선 타이타닉은 먼지 구름 속에 숨겨진 시공간 변칙과 충돌했습니다.  
안타깝게도, 그것은 1000년 전의 이름과 비슷한 운명을 만났습니다. 비록 그 배는 온전하게 유지되었지만,  
거의 절반의 승객들은 다른 차원으로 옮겨졌습니다!  

승무원을 구조하고 잃어버린 승객을 되찾는 것을 돕기 위해,  
여러분은 우주선의 손상된 컴퓨터 시스템에서 복구된 기록을 사용하여  
어떤 승객들이 이상 현상에 의해 이송되었는지 예측해야 합니다.

그들을 구하고 역사를 바꾸도록 도와주세요!

</br>

### 📔코드북




[출처] <a href="https://www.kaggle.com/competitions/spaceship-titanic/overview/description">Spaceship Titanic</a>

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots